# 0- Introduction
This notebook runs a Supervised MLP, a self supervised VIME based on MLP and a semi-supervised VIME on MNIST dataset

In [1]:
import torch
from utils import mask_generator, pretext_generator
from data_loader import load_mnist_data
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from torch.utils.data import (
    Dataset,
    DataLoader,
)

In [2]:
torch.manual_seed(0)

In [3]:
from data_sets import TabLabDataset, TabSemiUnlabDataset, TabUnlabDataset, ConcatDataset

In [4]:
from vim_self_pytorch import VimPretext
from mlp import MLP


We use only a subset of 1000  examples as labeled and the rest as unlabeled

In [5]:
# load the data as numpy arrays first
label_data_rate = 0.1  # size of unlabeled = 90% * 60000 = 54000
p_m = 0.3 # binomial distribution parameter
x_train, y_train, x_unlab, x_test, y_test = load_mnist_data(label_data_rate)
label_no = 1000     
# Use subset of labeled data
x_train = x_train[:label_no, :]
y_train = y_train[:label_no, :]  

# 1- Self supervised VIME (working)

Let's corrupt the unlabeled data

In [6]:
# corrupt x_unlab
m_unlab = mask_generator(p_m, x_unlab)
m_label, x_tilde = pretext_generator(m_unlab, x_unlab)

In [7]:
x_train.shape, x_unlab.shape, x_tilde.shape

((1000, 784), (54000, 784), (54000, 784))

In [8]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Hyperparameters
learning_rate = 3e-4
batch_size = 128
num_epochs = 10
# Load Data
dataset = TabUnlabDataset(x_unlab, x_tilde, m_label)

In [9]:
#train_set, test_set = torch.utils.data.random_split(dataset, [int(0.7*x_unlab.shape[0]), x_unlab.shape[0] - int(0.7*x_unlab.shape[0])])
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last = True)
#test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True, drop_last = True)

Let's train the self supervised model

In [10]:
model = VimPretext()
model.to(device)
model.train()
loss_function = torch.nn.MSELoss()
# Loss and optimizer
criterion =torch.nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
# Train Network
for epoch in range(num_epochs):
    losses = []
    for batch_idx,(x, x_tilde, mask) in enumerate(train_loader):
        # Get data to cuda if possible
        x = x.squeeze().float().to(device=device)
        x_tilde = x_tilde.squeeze().float().to(device=device)
        mask = mask.squeeze().float().to(device=device)
        # forward
        mask_lab, feature, encoder = model(x_tilde.float())
        loss_rec = loss_function(x.float(), feature)
        loss_mask = criterion(mask_lab,mask.float())
        loss = 2 * loss_rec + loss_mask
        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

Cost at epoch 0 is 0.34195022465497465
Cost at epoch 1 is 0.2716918497074245
Cost at epoch 2 is 0.2577599500463015
Cost at epoch 3 is 0.25018959618096115
Cost at epoch 4 is 0.24577393526136168
Cost at epoch 5 is 0.24334340629271828
Cost at epoch 6 is 0.2419069013303929
Cost at epoch 7 is 0.2408806901433972
Cost at epoch 8 is 0.24011515969335326
Cost at epoch 9 is 0.2394396678993651


In [11]:
torch.save(model.state_dict(), "self_vim.pth")

In [12]:
model.eval()

VimPretext(
  (fc0): Sequential(
    (0): Linear(in_features=784, out_features=784, bias=True)
    (1): ReLU()
  )
  (fc1): Sequential(
    (0): Linear(in_features=784, out_features=784, bias=True)
    (1): Sigmoid()
  )
  (fc2): Sequential(
    (0): Linear(in_features=784, out_features=784, bias=True)
    (1): Sigmoid()
  )
)

# 2 - Semi-Supervised VIME (not working correctly)

We will train on two datasets jointly : the labeled and the unlabeled that's why we will use ConcatDataset class

The semi supervised training doesn't work as expected. So I still need to debug this part. I'm suspecting the problem comes from the dataloader

In [13]:
# Load Data
train_data = TabLabDataset(x_train, y_train)
test_data = TabLabDataset(x_test, y_test)
unlab_data = TabSemiUnlabDataset(x_unlab)

In [14]:
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i %len(d)] for d in self.datasets)

    def __len__(self):
        return max(len(d) for d in self.datasets)

In [15]:
train_loader = torch.utils.data.DataLoader(
             ConcatDataset(
                 train_data,
                 unlab_data
             ),
             batch_size=128, shuffle=True, drop_last = True)

In [16]:
test_loader = torch.utils.data.DataLoader(test_data,
             batch_size=128, shuffle=False, drop_last = True)

Inspect the elements of the train_loader : 

In [17]:
for i, t in enumerate(train_loader):
    print(t[0][0].squeeze().shape)
    print(t[0][1].squeeze().shape)
    print(t[1].squeeze().shape)
    break

torch.Size([128, 784])
torch.Size([128, 10])
torch.Size([128, 784])


Let's train the network in a semi supervised fashion

In [29]:
predictor = MLP(28* 28, 10)

In [30]:
predictor = predictor.to(device)
predictor.train()
loss_function = torch.nn.MSELoss()
# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(predictor.parameters(), lr=learning_rate, weight_decay=1e-5)

# Train Network
for epoch in range(5):
    losses = []

    for batch_idx,(t) in enumerate(train_loader):
        # Get data to cuda if possible
        x_train = t[0][0].squeeze().float().to(device=device)
        y_train = t[0][1].squeeze().float().to(device=device)
        x_unlab_tr = t[1].squeeze().float().to(device=device)

        # encode labeled data
        x_encoded = model(torch.tensor(x_train).float())[-1]
        y_hat = predictor(x_encoded)
        # supervised loss
        loss_sup = criterion(y_hat,y_train)


        res = torch.empty(size=(10,128, 10))
        for i in range(10):
            # corrupt x_unlab
            m_unlab = mask_generator(p_m, x_unlab_tr)
            m_label, x_tilde = pretext_generator(m_unlab, x_unlab_tr)
            x_tilde_encoded = model(torch.tensor(x_tilde).float())[-1]
            y_unlab_hat = predictor(x_tilde_encoded)
            res[i] = y_unlab_hat
        # unsupervised loss
        loss_unsup = torch.var(res, dim = 0).mean()
        loss = loss_sup + loss_unsup
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

C:\Users\Salma El Ghourbal\AppData\Local\Temp\ipykernel_16440\3751243389.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_encoded = model(torch.tensor(x_train).float())[-1]
C:\Users\Salma El Ghourbal\AppData\Local\Temp\ipykernel_16440\3751243389.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_tilde_encoded = model(torch.tensor(x_tilde).float())[-1]


Cost at epoch 0 is 0.008831759246681196
Cost at epoch 1 is 0.008419817433008955
Cost at epoch 2 is 0.008242735599395498
Cost at epoch 3 is 0.00805914340067363
Cost at epoch 4 is 0.007804028658890951


# 3 - Supervised MLP for benchmark (working)

We need to train a supervised MLP on the labeled data : 

In [20]:
supervised = MLP(28* 28, 10)

In [21]:
supervised = supervised.to(device)
supervised.train()
# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(supervised.parameters(), lr=learning_rate, weight_decay=1e-5)

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx,(t) in enumerate(train_loader):
        # Get data to cuda if possible
        x_train = t[0][0].squeeze().float().to(device=device)
        y_train = t[0][1].squeeze().float().to(device=device)

        y_hat = supervised(x_train)
        loss = criterion(y_hat,y_train)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")



Cost at epoch 0 is 0.0034739986004533505
Cost at epoch 1 is 0.001042267829710684
Cost at epoch 2 is 0.0006413909726507405
Cost at epoch 3 is 0.00045295590310824276
Cost at epoch 4 is 0.0003423325419083078
Cost at epoch 5 is 0.0002771728671948932
Cost at epoch 6 is 0.00023589939671383856
Cost at epoch 7 is 0.00021006901754838395
Cost at epoch 8 is 0.00019176872297120445
Cost at epoch 9 is 0.00017821644908804423


We will train a supervised MLP using the encoding given by the self supervised training approach :

In [22]:
self_supervised = MLP(28* 28, 10)
self_supervised = self_supervised.to(device)
self_supervised.train()
# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(self_supervised.parameters(), lr=learning_rate, weight_decay=1e-5)

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx,(t) in enumerate(train_loader):
        # Get data to cuda if possible
        x_train = t[0][0].squeeze().float().to(device=device)
        y_train = t[0][1].squeeze().float().to(device=device)

        # encode x_train
        
        x_encoded = model(torch.tensor(x_train).float())[-1]
        y_hat = self_supervised(x_encoded)
        loss = criterion(y_hat,y_train)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")



C:\Users\Salma El Ghourbal\AppData\Local\Temp\ipykernel_16440\3389353923.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_encoded = model(torch.tensor(x_train).float())[-1]


Cost at epoch 0 is 0.0023966449176817425
Cost at epoch 1 is 0.0007000746084609447
Cost at epoch 2 is 0.00038063547929589615
Cost at epoch 3 is 0.00023242917058811325
Cost at epoch 4 is 0.00016305450120672027
Cost at epoch 5 is 0.00012749763989376985
Cost at epoch 6 is 0.00010843933615604008
Cost at epoch 7 is 9.593825319017595e-05
Cost at epoch 8 is 8.787968083776449e-05
Cost at epoch 9 is 8.148762042380619e-05


# Final comparison

Now Let's compare the performances of the three following models on the same test set : <br>
    - Supervised MLP <br>
    - Self supervised only MLP with VIME approach <br>
    - Semi-supervised MLP with VIME approach

In [23]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.squeeze().float().to(device=device)
            y = y.squeeze().float().to(device=device)
            

            scores = model(x.float())
            _, predictions = scores.max(1)
            num_correct += (predictions == y.argmax(-1)).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [24]:
# Check accuracy on training & test to see how good our model
def check_accuracy_self(loader, model, encoder):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.squeeze().float().to(device=device)
            y = y.squeeze().float().to(device=device)
            x_encoded = encoder(torch.tensor(x).float())[-1]
            scores = model(x_encoded.float())
            _, predictions = scores.max(1)
            num_correct += (predictions == y.argmax(-1)).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [25]:
# Check accuracy on training & test to see how good our model
def check_accuracy_semi(loader, model, encoder):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.squeeze().float().to(device=device)
            y = y.squeeze().float().to(device=device)
            x_encoded = encoder(torch.tensor(x).float())[-1]
            scores = model(x_encoded.float())
            _, predictions = scores.max(1)
            num_correct += (predictions == y.argmax(-1)).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [31]:
print(f"Accuracy on test set for the semi supervised approach: {check_accuracy_semi(test_loader, predictor,model)*100:.2f}")
print(f"Accuracy on test set for the supervised only approach: {check_accuracy(test_loader, supervised)*100:.2f}")
print(f"Accuracy on test set for the self supervised approach: {check_accuracy_self(test_loader, self_supervised, model)*100:.2f}")


C:\Users\Salma El Ghourbal\AppData\Local\Temp\ipykernel_16440\1829281224.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_encoded = encoder(torch.tensor(x).float())[-1]


Accuracy on test set for the semi supervised approach: 79.85
Accuracy on test set for the supervised only approach: 88.37
Accuracy on test set for the self supervised approach: 90.26
